In this notebook, we will list various techniques in dealing with missing and unorderly data.

This is of imperative importance before running ML algo's. In fact, certain ML algo's will not run if values are missing or NaN. E.g. LDA!

Source of data: https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data

The pima dataset is known to have missing values that are represented by a 0 value - i.e. bmi/age

Important Caveat: It could be that missing data has a meaning! If so you would need to treat this as a feature - e.g. dummy variable!

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
diabetes = pd.read_csv("pima-indians-diabetes.data.csv", header=None)

In [3]:
cols = ["Number of times pregnant.",
"Plasma glucose concentration a 2 hours in an oral glucose tolerance test.",
"Diastolic blood pressure (mm Hg).",
"Triceps skinfold thickness (mm).",
"2-Hour serum insulin (mu U/ml).",
"Body mass index (weight in kg/(height in m)^2).",
"Diabetes pedigree function.",
"Age (years).",
"Class variable (0 or 1)."] #obtained from UCI website. Simple copy paste.

reference = {}
counter = 0
for i in cols:
    if i not in reference.keys():
        reference[i] = counter
        counter +=1
final_reference = {v:k for k, v in reference.items()} #this is so cute - i was very proud of this dict comp.

In [4]:
final_reference

{0: 'Number of times pregnant.',
 1: 'Plasma glucose concentration a 2 hours in an oral glucose tolerance test.',
 2: 'Diastolic blood pressure (mm Hg).',
 3: 'Triceps skinfold thickness (mm).',
 4: '2-Hour serum insulin (mu U/ml).',
 5: 'Body mass index (weight in kg/(height in m)^2).',
 6: 'Diabetes pedigree function.',
 7: 'Age (years).',
 8: 'Class variable (0 or 1).'}

Above, I have created a dictionary that can quickly take an index of a column and return it;s description. I omitted the full column names from the dataframe for simplicity.

In [5]:
diabetes.head(1)

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1


In [6]:
for i in diabetes.columns:
    print(i, (sum(pd.isnull(diabetes[i]))/diabetes.shape[0])*100)

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


The above code tell's us that we have no null values in our dataset. This is to be expected, as we know our missing/corrupt data will be represented as an integer of value 0. We have to investigate further!

In [7]:
diabetes.describe()

,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Fantastic - the above shows that some columns have a min value of 0. This doesnt really make any sense! looks like we have some corrupt/missing data!

It is reasonable to assume that the following features should not have a 0 value:

- plasma glucose
- blood pressure
- skinfold thickness
- insulin
- BMI

Let's get a feel for how much data we have and how much might be corrupt!

In [8]:
diabetes.shape

(768, 9)

In [9]:
((diabetes[[1,2,3,4,5]] == 0).sum()/diabetes.shape[0])*100 #missing data in % terms.

1     0.651042
2     4.557292
3    29.557292
4    48.697917
5     1.432292
dtype: float64

From the above we can see that our missing data is quite spread out. As such, we will take several approached to ensure that we keep as much data as possible instead of just dropping rows. We want to retain as much data as possible because this will be very important in training our predictive model!

The first step is to replace these istaken 0 values with NaN's - this can be done easily with numpy!

In [10]:
diabetes[[1,2,3,4,5]] = diabetes[[1,2,3,4,5]].replace(0, np.NaN)

In [11]:
(diabetes.isnull().sum()/diabetes.shape[0])*100

0     0.000000
1     0.651042
2     4.557292
3    29.557292
4    48.697917
5     1.432292
6     0.000000
7     0.000000
8     0.000000
dtype: float64

We can see that we have obtained identical %'s as before. This means we replaced the correct data!

In [12]:
diabetes.head() #sweet, we see NaN's

,0,1,2,3,4,5,6,7,8
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [13]:
diabetes.shape

(768, 9)

There are several ways to deal with missing data - I will go through 3 of the most common:

1. remove all rows with missing data - this will severly reduce data at times! Pretty good strategy if you have a TON of data.

2. Impute the data

Let's start with removing rows!

In [14]:
row_example = diabetes.copy(deep=True) #making a copy

In [15]:
row_example.dropna(inplace=True)

In [16]:
row_example.shape #went from 768 rows to 392! Loss of around 48% of data!!

(392, 9)

In [17]:
(row_example.isnull().sum()/row_example.shape[0])*100 #we see all NaN's have been removed

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
dtype: float64

In [18]:
impute_example = diabetes.copy(deep=True)

In [19]:
impute_example.fillna(impute_example.mean(), inplace=True)

,0,1,2,3,4,5,6,7,8
0,6,148.0,72.000000,35.00000,155.548223,33.600000,0.627,50,1
1,1,85.0,66.000000,29.00000,155.548223,26.600000,0.351,31,0
2,8,183.0,64.000000,29.15342,155.548223,23.300000,0.672,32,1
3,1,89.0,66.000000,23.00000,94.000000,28.100000,0.167,21,0
4,0,137.0,40.000000,35.00000,168.000000,43.100000,2.288,33,1
5,5,116.0,74.000000,29.15342,155.548223,25.600000,0.201,30,0
6,3,78.0,50.000000,32.00000,88.000000,31.000000,0.248,26,1
7,10,115.0,72.405184,29.15342,155.548223,35.300000,0.134,29,0
8,2,197.0,70.000000,45.00000,543.000000,30.500000,0.158,53,1
9,8,125.0,96.000000,29.15342,155.548223,32.457464,0.232,54,1


In [20]:
(impute_example.isnull().sum()/impute_example.shape[0])*100 #All NaN's have been removed!

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
dtype: float64

Another way to impute is to Use the Sklearn library! I usually use this when I am preparing to run an algorithm. Let's run an LDA on this example as I mentioned at the beginning. LDA wouldnt run on missing values, however, since we have taken care of that - let's give it a shot!

In [21]:
X = diabetes.iloc[:,0:8].values
y = diabetes.iloc[:, 8].values

impute = Imputer()

X = impute.fit_transform(X) #defaults to replacing NaN's with the mean of the columns (axis=0)

Be sure to fit and transform the X variable. Let's go ahead run the LDA model!

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
test_size=0.3)

In [23]:
algo = LinearDiscriminantAnalysis()
kfolds = KFold(n_splits=10)
metrics = cross_val_score(algo, X_train, y_train, cv=kfolds, scoring='accuracy')
print(metrics.mean()) #pretty good for training!

0.770684835779


In [24]:
LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train, y_train)
predictions = LDA.predict(X_test)

In [25]:
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.766233766234
[[133  18]
 [ 36  44]]
             precision    recall  f1-score   support

          0       0.79      0.88      0.83       151
          1       0.71      0.55      0.62        80

avg / total       0.76      0.77      0.76       231



Awesome!